# ウェブサイトのAPIからのJSON資料をCSVに変換する。

Source data from a website, which has lot of online classes with their information.  
API site hidden address [API Location](https://api.hahow.in/api/courses?limit=24&page=0)  
We first try to maxmized the per item in 1 pg. Err Msg found as :  
 -   `{"name":"ValidationError","message":"child \"limit\" fails because [\"limit\" must be less than or equal to 30]","errorCode":20001}`  
  
So the max per time can only be trace from **30**/per pg.

In [1]:
# Using requests to get the information from web pg.
import requests
#Might needs to add lib of csv and json
import json
import csv

#### Get data from web.
Origional json format.  
Using `.json()` method for quick pretty look and ensure what kind of data wants to extract.  
Using `.text()` method if wants to translate to text only.  

In [2]:
raw_data = requests.get('https://api.hahow.in/api/courses?limit=1&page=0')
raw_data.json()

{'_metadata': {'page': 0, 'count': 465},
 'data': [{'successCriteria': {'numSoldTickets': 100},
   'campaign': {'types': []},
   'numSoldTickets': 30,
   'status': 'INCUBATING',
   'reviewing': False,
   'price': 2500,
   'preOrderedPrice': 1688,
   'type': 'COURSE',
   'isReject': False,
   'averageRating': 0,
   'numRating': 0,
   '_id': '5eaf7eec3e47cfd04f222519',
   'owner': {'_id': '57313363b2a279090045fca1',
    'profileImageUrl': 'https://graph.facebook.com/10154199592344375/picture',
    'name': '李柏鋒',
    'username': 'firefly'},
   'uniquename': 'writing-course',
   'estimatedCourseStartTime': '2020-08-09T16:00:00.000Z',
   'incubateTime': '2020-05-12T04:00:20.496Z',
   'proposalDueTime': '2020-06-10T16:00:00.000Z',
   'categories': [],
   'modules': ['5eafcad34dec4b13da1e0f09',
    '5eaf83868f9341dd44eb465a',
    '5eafcb72be3502e9d0fc59cc',
    '5eafcb7d94c64e2d49b96405',
    '5eafcb9194c64e32e2b96406'],
   'title': '職場寫作課：從個人品牌到自媒體',
   'metaDescription': '寫作是讓職場上的專業人士被看見的最重

Extract from `.json` file first.

In [3]:
raw_data = requests.get('https://api.hahow.in/api/courses?limit=30&page=0')
# check on first `0` record to consider which data needs to be extract.
raw_data.json()['data'][0]

{'successCriteria': {'numSoldTickets': 100},
 'campaign': {'types': []},
 'numSoldTickets': 30,
 'status': 'INCUBATING',
 'reviewing': False,
 'price': 2500,
 'preOrderedPrice': 1688,
 'type': 'COURSE',
 'isReject': False,
 'averageRating': 0,
 'numRating': 0,
 '_id': '5eaf7eec3e47cfd04f222519',
 'owner': {'_id': '57313363b2a279090045fca1',
  'profileImageUrl': 'https://graph.facebook.com/10154199592344375/picture',
  'name': '李柏鋒',
  'username': 'firefly'},
 'uniquename': 'writing-course',
 'estimatedCourseStartTime': '2020-08-09T16:00:00.000Z',
 'incubateTime': '2020-05-12T04:00:20.496Z',
 'proposalDueTime': '2020-06-10T16:00:00.000Z',
 'categories': [],
 'modules': ['5eafcad34dec4b13da1e0f09',
  '5eaf83868f9341dd44eb465a',
  '5eafcb72be3502e9d0fc59cc',
  '5eafcb7d94c64e2d49b96405',
  '5eafcb9194c64e32e2b96406'],
 'title': '職場寫作課：從個人品牌到自媒體',
 'metaDescription': '寫作是讓職場上的專業人士被看見的最重要方式，從打造個人品牌最重要的 10 篇關鍵文章，到發展成具有影響力的自媒體，本課程幫助學員被認識、被重視、被賞識。',
 'coverImage': {'_id': '5eba2373ead8a45591dc

In [4]:
#extract the wanted information.
cl_pri = raw_data.json()['data'][0]['price']
teach = raw_data.json()['data'][0]['owner']['name']
cor_tit = raw_data.json()['data'][0]['title']
cor_des = raw_data.json()['data'][0]['metaDescription']
curre = raw_data.json()['data'][0]['currencyCode']
print(cl_pri, curre, teach, cor_tit, cor_des)

2500 TWD 李柏鋒 職場寫作課：從個人品牌到自媒體 寫作是讓職場上的專業人士被看見的最重要方式，從打造個人品牌最重要的 10 篇關鍵文章，到發展成具有影響力的自媒體，本課程幫助學員被認識、被重視、被賞識。


 - Then we knows what we wants, the next step is pretty output as `.csv` file.
 - And we needs to use file `open` method.

In [5]:
with open('./raw/course_output.csv', 'w', newline='' ,encoding="utf-8") as out_csv:
          writer = csv.writer(out_csv)
          #we needs to put title in first line.
          title = "費用", "両替率", "先生名", "授業名", "授業説明"
          writer.writerow(title)
          for item in raw_data.json()['data']:
                    cl_pri = item['price']
                    teach = item['owner']['name']
                    cor_tit = item['title']
                    cor_des = item['metaDescription']
                    curre = item['currencyCode']
                    row_data = cl_pri, curre, teach, cor_tit, cor_des
                    writer.writerow(row_data)

>  We can take a quick peak of exported `csv` file using panda.

In [6]:
!pip3 install pandas
import pandas as pd
pd.read_csv('./raw/course_output.csv',encoding="utf-8")

,費用,両替率,先生名,授業名,授業説明
0,2500,TWD,李柏鋒,職場寫作課：從個人品牌到自媒體,寫作是讓職場上的專業人士被看見的最重要方式，從打造個人品牌最重要的 10 篇關鍵文章，到發展...
1,1200,TWD,Yo Chen,新鮮人用設計思考寫出好履歷,寫履歷也要學？不要等到收不到通知才開始，好履歷也不是只有經歷豐富人的專利，這堂課彙整超過百位...
2,2600,TWD,連禾,紙上的美妝師：連禾時尚彩妝畫,時尚彩妝設計圖主要是以人體上半身的臉部及髮型設計紙圖為主，是傳達原創構思最佳方式，更是一項提...
3,2388,TWD,臺灣行銷研究 - Ivan,顧客分類大師：Python x RFM 會員經營新觀點,☛ 不知如何分析顧客的購買紀錄嗎？\n\n☛ 如何區分新顧客、常貴客、沉睡客、流失客？\n\...
4,2100,TWD,Lulu🌸,翻轉文字：給孩子好玩的創意寫作課！,創意有趣的寫作課，帶領學生思辨讀寫，提升閱讀理解能力；用輔導的角度，啟發學生的想像、引導寫出...
5,1900,TWD,李李宜亞,大人的蠟筆畫：活用油畫知識，寫實生活微光,蠟筆畫入門課程，教你活用油畫知識，畫出寫實蠟筆畫。課程將從媒材介紹開始，教你蠟筆基礎繪畫技法...
6,2700,TWD,TJ,機器學習攻略：實戰心法到求職技巧,從什麼是機器學習開始，教你使用 scikit-learn 以及 PyTorch 訓練模型。接...
7,3480,TWD,菓實日,法式甜點在地魂製作攻略－日日在家菓實日,從台灣物產與多元文化跟法式甜點的關係到在家中如何製作甜點、節省原料、變化食材，知名在地法式甜...
8,2400,TWD,職感・加,林長揚職場簡報術：一次搞定所有工作簡報！,職場簡報怎麼做？長揚老師推出最新職場簡報課程，內容囊括業務提案、會議報告、書面簡報，幫你打通...
9,2880,TWD,kellykiwi,概念式時尚插畫｜我的小小時裝週,好多喜歡的衣服，鞋子包包，沒有辦法全部帶回家，畫下來也是很過癮的，利用水彩和色鉛筆，一個一個...
